In [4]:
import polars as pl
import optuna
from catboost import CatBoostRegressor
import numpy as np

from src.util.constants import DATA_PATH, META_MODEL_PERFORMANCE, FIXED_CB_PARAMETERS
from src.util.common import load_from_pickle, mean_grouped_spearman_correlation, save_as_pickle

In [5]:
selected_features = load_from_pickle(DATA_PATH / 'results/selected_features.pkl')
required_columns = ['era', 'target'] + selected_features

df_train_list = []
df_validate_list = []
for fold in range(2):
    df_train_fold = pl.read_parquet(f"{DATA_PATH}/folds/df_train_{fold}.parquet")
    df_validate_fold = pl.read_parquet(f"{DATA_PATH}/folds/df_validate_{fold}.parquet")

    df_train_fold = df_train_fold.select(required_columns)
    df_validate_fold = df_validate_fold.select(required_columns)

    df_train_list.append(df_train_fold)
    df_validate_list.append(df_validate_fold)
    del df_train_fold, df_validate_fold


df_meta_model = pl.read_parquet(f'{DATA_PATH}/folds/df_meta_model.parquet')
df_meta_model = df_meta_model.select(required_columns + ['numerai_meta_model'])

In [6]:
def performance_approximation(model: CatBoostRegressor, fold: int) -> tuple[float, float, float, float]:
    df_validate_fold = df_validate_list[fold]

    corr = mean_grouped_spearman_correlation(
        pl.Series(model.predict(df_validate_fold[selected_features].to_pandas())),
        df_validate_fold['target'],
        df_validate_fold['era']
    )

    df_mm_corr = pl.DataFrame({
        'prediction': pl.Series(model.predict(df_meta_model[selected_features].to_pandas())),
        'numerai_meta_model': df_meta_model['numerai_meta_model'],
        'era': df_meta_model['era']
    })
    corr_w_mm = df_mm_corr.select(
        pl.corr("prediction", "numerai_meta_model", method="spearman")
        .over('era', mapping_strategy='explode')
    ).mean().item()

    mmc_approximation = corr - corr_w_mm * META_MODEL_PERFORMANCE[fold]
    performance = .75 * corr + 2.25 * mmc_approximation

    return performance, corr, corr_w_mm, mmc_approximation

In [7]:
def objective(trial: optuna.trial.Trial) -> float:
    # we use these exponents to create a log scale that includes zero
    l2_leaf_reg_exponent = trial.suggest_int("l2_leaf_reg_exponent", 0, 10)
    parameter_sample = {
        "learning_rate": trial.suggest_float("learning_rate", .001, .2, log=True),
        "iterations": trial.suggest_int("iterations", 10, 5000),
        'border_count': trial.suggest_int('border_count', 2 ** 3 - 1, 2 ** 10 - 1, log=True),
        "depth": trial.suggest_int("depth", 2, 12),
        "subsample": trial.suggest_float("subsample", .1, 1.0),
        "rsm": trial.suggest_float("rsm", .1, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 500000, log=True),
        "l2_leaf_reg": (10 ** l2_leaf_reg_exponent - 1) / 10 ** 5,
    }

    parameters = {
        **FIXED_CB_PARAMETERS,
        **parameter_sample
    }

    result_performance = []

    for fold in range(2):
        model = CatBoostRegressor(**parameters)
        model.fit(
            X=df_train_list[fold][selected_features].to_pandas(),
            y=df_train_list[fold]['target'].to_pandas()
        )

        performance, corr, corr_w_mm, mmc_approximation = performance_approximation(model, fold)
        trial.set_user_attr(f"performance_{fold}", performance)
        trial.set_user_attr(f"correlation_{fold}", corr)
        trial.set_user_attr(f"correlation_with_meta_model_{fold}", corr_w_mm)
        trial.set_user_attr(f"mmc_approximation_{fold}", mmc_approximation)
        print(f'Results for fold {fold}: Performance {performance:.5f}, correlation: {corr:.5f}, correlation with meta model: {corr_w_mm:.5f}, MMC (approx.): {mmc_approximation:.5f}')

        # stop trial if it does not show promise (by default, median pruner is used)
        trial.report(performance, step=fold)
        if fold == 0 and trial.number > 10 and trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        result_performance.append(performance)

    return np.mean(result_performance)

In [8]:
study = optuna.create_study(direction="maximize")

[I 2026-01-28 00:25:12,058] A new study created in memory with name: no-name-bc357e79-8d4b-4bfb-8b99-15d8c4a2a4bd


In [ ]:
# cumulative performance 50 runs:
# best performance: 0.0479
# runtime: 27:25

In [15]:
study.optimize(objective, n_trials=19, show_progress_bar=True)

Best trial: 17. Best value: 0.0479147:   5%|▌         | 1/19 [22:25<6:43:41, 1345.63s/it]

Results for fold 0: Performance 0.04005, correlation: 0.02732, correlation with meta model: 0.42335, MMC (approx.): 0.00869
[I 2026-01-29 00:56:48,543] Trial 26 pruned. 


Best trial: 17. Best value: 0.0479147:  11%|█         | 2/19 [47:32<6:48:07, 1440.41s/it]

Results for fold 0: Performance 0.04398, correlation: 0.03411, correlation with meta model: 0.58943, MMC (approx.): 0.00818
[I 2026-01-29 01:21:55,306] Trial 27 pruned. 
Results for fold 0: Performance 0.05115, correlation: 0.03719, correlation with meta model: 0.61023, MMC (approx.): 0.01034


Best trial: 17. Best value: 0.0479147:  16%|█▌        | 3/19 [1:21:02<7:33:26, 1700.42s/it]

...

Results for fold 1: Performance 0.03678, correlation: 0.02902, correlation with meta model: 0.60400, MMC (approx.): 0.00667
[I 2026-01-29 09:45:30,971] Trial 42 finished with value: 0.04466589803117406 and parameters: {'l2_leaf_reg_exponent': 6, 'learning_rate': 0.02718706838146236, 'iterations': 4698, 'border_count': 275, 'depth': 4, 'subsample': 0.18572227113299933, 'rsm': 0.47729305680195605, 'min_data_in_leaf': 211562}. Best is trial 17 with value: 0.04791472124516886.


Best trial: 17. Best value: 0.0479147:  95%|█████████▍| 18/19 [9:23:14<24:32, 1472.79s/it]

Results for fold 0: Performance 0.04849, correlation: 0.03322, correlation with meta model: 0.51689, MMC (approx.): 0.01048
[I 2026-01-29 09:57:37,330] Trial 43 pruned. 


Best trial: 17. Best value: 0.0479147: 100%|██████████| 19/19 [9:36:21<00:00, 1820.08s/it]

Results for fold 0: Performance 0.04715, correlation: 0.03348, correlation with meta model: 0.53822, MMC (approx.): 0.00980
[I 2026-01-29 10:10:44,451] Trial 44 pruned. 


In [16]:
# visualise result
for param in study.best_params.keys():
    optuna.visualization.plot_slice(study, params=[param]).show()

In [17]:
save_as_pickle(study, DATA_PATH / 'results/study_cb.pkl')